<a href="https://colab.research.google.com/github/Lynxlazy/genomics-data-analysis/blob/main/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!wget --output-document sratoolkit.tar.gz https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-ubuntu64.tar.gz
!tar -vxzf sratoolkit.tar.gz


--2023-03-03 07:59:06--  https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-ubuntu64.tar.gz
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 130.14.250.7, 130.14.250.10, 2607:f220:41f:250::230, ...
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|130.14.250.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86400775 (82M) [application/x-gzip]
Saving to: ‘sratoolkit.tar.gz’

sratoolkit.tar.gz   100%[===================>]  82.40M   145MB/s    in 0.6s    

2023-03-03 07:59:07 (145 MB/s) - ‘sratoolkit.tar.gz’ saved [86400775/86400775]

sratoolkit.3.0.1-ubuntu64/
sratoolkit.3.0.1-ubuntu64/README.md
sratoolkit.3.0.1-ubuntu64/schema/
sratoolkit.3.0.1-ubuntu64/schema/insdc/
sratoolkit.3.0.1-ubuntu64/schema/insdc/sra.vschema
sratoolkit.3.0.1-ubuntu64/schema/insdc/insdc.vschema
sratoolkit.3.0.1-ubuntu64/schema/insdc/seq.vschema
sratoolkit.3.0.1-ubuntu64/schema/align/
sratoolkit.3.0.1-ubuntu64/schema/align/refseq

In [8]:
!export PATH=$PATH:$PWD/sratoolkit.3.0.1-ubuntu64/bin
!which fastq-dump

# 1. Exploring the dataset

In [16]:
!./sratoolkit.3.0.1-ubuntu64/bin/fasterq-dump SRR292678
!./sratoolkit.3.0.1-ubuntu64/bin/fasterq-dump SRR292862
!./sratoolkit.3.0.1-ubuntu64/bin/fasterq-dump SRR292770

spots read      : 6,111,112
reads read      : 12,222,224
reads written   : 12,222,224
spots read      : 5,102,041
reads read      : 10,204,082
reads written   : 10,204,082
spots read      : 5,102,041
reads read      : 10,204,082
reads written   : 10,204,082


# 2. K-mer profile and genome size estimation


In [17]:
!sudo apt install jellyfish

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libhts3 libjellyfish-2.0-2
The following NEW packages will be installed:
  jellyfish libhts3 libjellyfish-2.0-2
0 upgraded, 3 newly installed, 0 to remove and 22 not upgraded.
Need to get 771 kB of archives.
After this operation, 1,727 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 libhts3 amd64 1.10.2-3ubuntu0.1 [350 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libjellyfish-2.0-2 amd64 2.3.0-4build1 [60.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 jellyfish amd64 2.3.0-4build1 [361 kB]
Fetched 771 kB in 0s (2,706 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 3.)
debcon

In [19]:
!wget https://trace.ncbi.nlm.nih.gov/Traces/sra-reads-be/fasta?acc=SRR292678

--2023-03-03 08:25:14--  https://trace.ncbi.nlm.nih.gov/Traces/sra-reads-be/fasta?acc=SRR292678
Resolving trace.ncbi.nlm.nih.gov (trace.ncbi.nlm.nih.gov)... 130.14.29.113, 2607:f220:41e:4290::113
Connecting to trace.ncbi.nlm.nih.gov (trace.ncbi.nlm.nih.gov)|130.14.29.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘fasta?acc=SRR292678’

fasta?acc=SRR292678     [           <=>      ] 438.04M  1.09MB/s    in 6m 44s  

2023-03-03 08:31:59 (1.08 MB/s) - ‘fasta?acc=SRR292678’ saved [459321528]



In [21]:
!mv fasta?acc=SRR292678 SRR292678.fasta

In [22]:
!jellyfish count -m 21 -s 5G -t 10 -C SRR292678_1.fastq

terminate called after throwing an instance of 'std::runtime_error'
  what():  Unsupported format


# 3. Assembling E. coli X genome from paired reads

In [20]:
!apt -y install spades

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  bamtools bwa cwltool fonts-glyphicons-halflings libbamtools2.5.1 libc-ares2
  libhat-trie0 libjs-bootstrap libnlopt0 libnode64 libssw0 nodejs nodejs-doc
  python3-atomicwrites python3-attr python3-bs4 python3-cachecontrol
  python3-coloredlogs python3-dateutil python3-decorator python3-html5lib
  python3-humanfriendly python3-importlib-metadata python3-isodate
  python3-joblib python3-lockfile python3-lxml python3-mistune
  python3-more-itertools python3-msgpack python3-mypy-extensions
  python3-networkx python3-packaging python3-pluggy python3-prov
  python3-psutil python3-py python3-pyparsing python3-pytest python3-rdflib
  python3-rdflib-jsonld python3-ruamel.yaml python3-schema-salad
  python3-shellescape python3-simplejson python3-soupsieve
  python3-sparqlwrapper python3-typing-extensions python3-wcwidth
  python3-webencodings pyth

In [23]:
!mkdir result

In [25]:
!spades --test

Command line: /usr/lib/spades/bin/spades.py	--test	

System information:
  SPAdes version: 3.13.1
  Python version: 3.8.10
  OS: Linux-5.10.147+-x86_64-with-glibc2.29

Output dir: /content/spades_test
Mode: read error correction and assembling
Debug mode is turned OFF

Dataset parameters:
  Multi-cell mode (you should set '--sc' flag if input data was obtained with MDA (single-cell) technology or --meta flag if processing metagenomic dataset)
  Reads:
    Library number: 1, library type: paired-end
      orientation: fr
      left reads: ['/usr/share/spades/test_dataset/ecoli_1K_1.fq.gz']
      right reads: ['/usr/share/spades/test_dataset/ecoli_1K_2.fq.gz']
      interlaced reads: not specified
      single reads: not specified
      merged reads: not specified
Read error correction parameters:
  Iterations: 1
  PHRED offset will be auto-detected
  Corrected reads will be compressed
Assembly parameters:
  k: automatic selection based on read length
  Repeat resolution is enabled
  Mis

In [26]:
!spades -1 SRR292678_1.fastq -2 SRR292678_2.fastq -o output

Command line: /usr/lib/spades/bin/spades.py	-1	/content/SRR292678_1.fastq	-2	/content/SRR292678_2.fastq	-o	/content/output	

System information:
  SPAdes version: 3.13.1
  Python version: 3.8.10
  OS: Linux-5.10.147+-x86_64-with-glibc2.29

Output dir: /content/output
Mode: read error correction and assembling
Debug mode is turned OFF

Dataset parameters:
  Multi-cell mode (you should set '--sc' flag if input data was obtained with MDA (single-cell) technology or --meta flag if processing metagenomic dataset)
  Reads:
    Library number: 1, library type: paired-end
      orientation: fr
      left reads: ['/content/SRR292678_1.fastq']
      right reads: ['/content/SRR292678_2.fastq']
      interlaced reads: not specified
      single reads: not specified
      merged reads: not specified
Read error correction parameters:
  Iterations: 1
  PHRED offset will be auto-detected
  Corrected reads will be compressed
Assembly parameters:
  k: automatic selection based on read length
  Repeat re

In [27]:
!pip install quast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for quast: filename=quast-5.2.0-py3-none-any.whl size=31206447 sha256=fe581ab34a1ec72852a46135a93952c82abbde029626a3d80adac06f022d3ade
  Stored in directory: /root/.cache/pip/wheels/79/fe/84/429b22fc6184f0807dfdcb67bdf0d95441b3415bce83219ad2
Successfully built quast


In [29]:
!quast.py output/contigs.fasta

/usr/local/bin/quast.py output/contigs.fasta

Version: 5.2.0

System information:
  OS: Linux-5.10.147+-x86_64-with-glibc2.29 (linux_64)
  Python version: 3.8.10
  CPUs number: 2

Started: 2023-03-03 11:27:07

Logging to /content/quast_results/results_2023_03_03_11_27_07/quast.log
NOTICE: Maximum number of threads is set to 1 (use --threads option to set it manually)

CWD: /content
Main parameters: 
  MODE: default, threads: 1, min contig length: 500, min alignment length: 65, min alignment IDY: 95.0, \
  ambiguity: one, min local misassembly length: 200, min extensive misassembly length: 1000

Contigs:
  Pre-processing...
  output/contigs.fasta ==> contigs

2023-03-03 11:27:08
Running Basic statistics processor...
  Contig files: 
    contigs
  Calculating N50 and L50...
    contigs, N50 = 105346, L50 = 15, auN = 127832.1, Total length = 5245723, GC % = 50.53, # N's per 100 kbp =  0.00
  Drawing Nx plot...
    saved to /content/quast_results/results_2023_03_03_11_27_07/basic_stats/Nx_

In [31]:
!cat quast_results/latest/report.txt

All statistics are based on contigs of size >= 500 bp, unless otherwise noted (e.g., "# contigs (>= 0 bp)" and "Total length (>= 0 bp)" include all contigs).

Assembly                    contigs 
# contigs (>= 0 bp)         510     
# contigs (>= 1000 bp)      147     
# contigs (>= 5000 bp)      82      
# contigs (>= 10000 bp)     69      
# contigs (>= 25000 bp)     51      
# contigs (>= 50000 bp)     33      
Total length (>= 0 bp)      5316686 
Total length (>= 1000 bp)   5206945 
Total length (>= 5000 bp)   5043449 
Total length (>= 10000 bp)  4953165 
Total length (>= 25000 bp)  4664641 
Total length (>= 50000 bp)  4002075 
# contigs                   201     
Largest contig              300784  
Total length                5245723 
GC (%)                      50.53   
N50                         105346  
N90                         21421   
auN                         127832.1
L50                         15      
L90                         54      
# N's per 100 kbp          

# 3a Effect of read correction.

# 4. Impact of reads with large insert size

In [ ]:
!spades.py --pe1-1 SRR292678_1.fastq --pe1-2 SRR292678_2.fastq \
    --mp1-1 SRR292770_1.fastq --mp1-2 SRR292770_2.fastq \
    --mp2-1 SRR292862_1.fastq --mp2-2 SRR292862_2.fastq \
    -o spades_output

Command line: /usr/lib/spades/bin/spades.py	--pe1-1	/content/SRR292678_1.fastq	--pe1-2	/content/SRR292678_2.fastq	--mp1-1	/content/SRR292770_1.fastq	--mp1-2	/content/SRR292770_2.fastq	--mp2-1	/content/SRR292862_1.fastq	--mp2-2	/content/SRR292862_2.fastq	-o	/content/spades_output	

System information:
  SPAdes version: 3.13.1
  Python version: 3.8.10
  OS: Linux-5.10.147+-x86_64-with-glibc2.29

Output dir: /content/spades_output
Mode: read error correction and assembling
Debug mode is turned OFF

Dataset parameters:
  Multi-cell mode (you should set '--sc' flag if input data was obtained with MDA (single-cell) technology or --meta flag if processing metagenomic dataset)
  Reads:
    Library number: 1, library type: mate-pairs
      orientation: rf
      left reads: ['/content/SRR292770_1.fastq']
      right reads: ['/content/SRR292770_2.fastq']
      interlaced reads: not specified
      single reads: not specified
      merged reads: not specified
    Library number: 2, library type: m

In [ ]:
!quast test_data/contigs_1.fasta \
               test_data/contigs_2.fasta \
               -r test_data/reference.fasta.gz \
               -g test_data/genes.gff

In [ ]:
!less quast_results/latest/report.txt

# 5. Genome Annotation

In [32]:
!sudo apt-get install libdatetime-perl libxml-simple-perl libdigest-md5-perl git default-jre bioperl
!sudo cpan Bio::Perl
!git clone https://github.com/tseemann/prokka.git $HOME/prokka
!$HOME/prokka/bin/prokka --setupdb

Выходные данные были обрезаны до нескольких последних строк (5000).
--------------------- WARNING ---------------------
MSG: No feature key in putative feature table line: FT   VARIANT         90

---------------------------------------------------

--------------------- WARNING ---------------------
MSG: failed to parse feature table line for seq NDP_HUMAN
FT   VARIANT         90

---------------------------------------------------

--------------------- WARNING ---------------------
MSG: No feature key in putative feature table line: FT                   /note="R -> P (in ND; dbSNP:rs104894867)"

---------------------------------------------------

--------------------- WARNING ---------------------
MSG: failed to parse feature table line for seq NDP_HUMAN
FT                   /note="R -> P (in ND; dbSNP:rs104894867)"

---------------------------------------------------

--------------------- WARNING ---------------------
MSG: No feature key in putative feature table line: FT        

In [34]:
!$HOME/prokka/bin/prokka

Name:
  Prokka 1.14.6 by Torsten Seemann <torsten.seemann@gmail.com>
Synopsis:
  rapid bacterial genome annotation
Usage:
  prokka [options] <contigs.fasta>
General:
  --help             This help
  --version          Print version and exit
  --citation         Print citation for referencing Prokka
  --quiet            No screen output (default OFF)
  --debug            Debug mode: keep all temporary files (default OFF)
Setup:
  --dbdir [X]        Prokka database root folders (default '/root/prokka/db')
  --listdb           List all configured databases
  --setupdb          Index all installed databases
  --cleandb          Remove all database indices
  --depends          List all software dependencies
Outputs:
  --outdir [X]       Output folder [auto] (default '')
  --force            Force overwriting existing output folder (default OFF)
  --prefix [X]       Filename output prefix [auto] (default '')
  --addgenes         Add 'gene' features for each 'CDS' feature (default OFF)
  --ad

# 6. Finding the closest relative of E. coli X

In [ ]:
!git clone https://github.com/tseemann/barrnap.git
cd barrnap/bin
./barrnap --help

# 7. What is the genetic cause of HUS?